In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.metrics

In [ ]:
data = []
folder = 'dashboard_logs'
for filename in os.listdir(folder):
    if 'flat' in filename:
        f = os.path.join(folder, filename)
        if os.path.isfile(f) and f[-4:] == '.csv':
            data.append((filename[:-4], pd.read_csv(f)))

In [ ]:
print('| Car           | run max speed | 0-75   | 0-100  | 0-120  | 0-150  | 15s AUC  | 5s AUC   |')
print('|---------------|---------------|--------|--------|--------|--------|----------|----------|')
data_plot = []
for car, df in data:
    # remove before start
    df = df[df.speed > 1]
    df.time = df.time - df.iloc[0]['time']
    
    print(f"|{car:>14} |"+
          f"{np.max(df.speed):>15}|"+
          f"{df.time[df['speed'] > 75].tolist()[0]/1000:>8}|"+
          f"{df.time[df['speed'] > 100].tolist()[0]/1000:>8}|"+
          f"{df.time[df['speed'] > 120].tolist()[0]/1000:>8}|"+
          f"{df.time[df['speed'] > 150].tolist()[0]/1000:>8}|", end='')
    
    # keep 15s
    df = df[df.time < 15000]
    
    time = df.time
    speed = df.speed.tolist()
    acceleration = df.speed.diff()
    time_rolling = df.time.rolling(20)
    acceleration_rolling = df.speed.diff().rolling(20)
    
    auc = sklearn.metrics.auc(df.time.tolist(), df.speed.tolist())
    print(f'{round(auc/10):>10}|', end='')
    
    # keep 5s
    df = df[df.time < 5000]
    
    auc = sklearn.metrics.auc(df.time.tolist(), df.speed.tolist())
    print(f'{round(auc/10):>10}|')
    
    # save
    data_plot.append((car, time, speed, acceleration, time_rolling, acceleration_rolling))

In [ ]:
fig = plt.figure(figsize=(16, 8))
for car, time, speed, acceleration, time_rolling, acceleration_rolling in data_plot:
    plt.plot(time, speed, label=car)
plt.ylim(0, 210)
plt.title('speed')
plt.legend(loc='best')
plt.savefig(folder + '/speed.png')
plt.show()

fig = plt.figure(figsize=(16, 8))
for car, time, speed, acceleration, time_rolling, acceleration_rolling in data_plot:
    plt.plot(time, speed, label=car)
plt.xlim(0, 5000)
plt.ylim(0, 150)
plt.title('speed 5s')
plt.legend(loc='best')
plt.savefig(folder + '/speed_5s.png')
plt.show()

fig = plt.figure(figsize=(16, 8))
for car, time, speed, acceleration, time_rolling, acceleration_rolling in data_plot:
    plt.plot(time, acceleration, label=car)
plt.title('acceleration')
plt.legend(loc='best')
plt.savefig(folder + '/acceleration.png')
plt.show()

fig = plt.figure(figsize=(16, 8))
for car, time, speed, acceleration, time_rolling, acceleration_rolling in data_plot:
    plt.plot(time_rolling.sum().tolist(), acceleration_rolling.sum().tolist(), label=car)
plt.title('acceleration smoothed')
plt.legend(loc='best')
plt.savefig(folder + '/acceleration_smoothed.png')
plt.show()